# exploring text summaries

 - does not work
 - we will only use the metadata

In [1]:
# import libraries
import urllib
from bs4 import BeautifulSoup
import re

In [2]:
from urllib.request import urlopen
content = urlopen("https://www.hoberg-driesch.de/")

In [3]:
soup = BeautifulSoup(content, 'html.parser')

In [4]:
def maystrip(a):
    if a is None:
        return a
    else:
        return a.strip()

In [5]:
a_text = soup.find_all('p')
#2 Removing
y=[re.sub(r'<.+?>',r'',str(a)) for a in a_text]

In [6]:
y

['\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tWir bündeln unser Know How!\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tMit der Übertragung der Handelsaktivitäten unserer Einzelgesellschaften zur Hoberg &amp; Driesch Röhrenhandel GmbH präsentiert sich die Röhrengruppe um die Hoberg &amp; Driesch GmbH &amp; Co. KG nun noch kundennäher und leistungsstärker.\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t',
 'Wir setzen höchste Qualitätsmaßstäbe in der Beratung und den Arbeitsabläufen.']

In [7]:
s = "\n".join(y)

In [8]:
a_text

[<p class="header-subline">
 								
 									Wir bündeln unser Know How!
 								
 							</p>, <p class="header-bodytext">
 								
 									Mit der Übertragung der Handelsaktivitäten unserer Einzelgesellschaften zur Hoberg &amp; Driesch Röhrenhandel GmbH präsentiert sich die Röhrengruppe um die Hoberg &amp; Driesch GmbH &amp; Co. KG nun noch kundennäher und leistungsstärker.
 								
 							</p>, <p>Wir setzen höchste Qualitätsmaßstäbe in der Beratung und den Arbeitsabläufen.</p>]

In [9]:
t_all = soup.get_text()

In [11]:
# kill all script and style elements
for script in soup(["script", "style"]):
    script.decompose()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

print(text)

HOBERG & DRIESCH - Hoberg und Driesch GmbH & Co. KG
Suchbegriff
Suchen
Downloads
Sprache wählen
deen
UnternehmenProdukteAnwen­dungs­gebieteServicesKontakt
Aus 6 mach 1
Wir bündeln unser Know How!
Mit der Übertragung der Handelsaktivitäten unserer Einzelgesellschaften zur Hoberg & Driesch Röhrenhandel GmbH präsentiert sich die Röhrengruppe um die Hoberg & Driesch GmbH & Co. KG nun noch kundennäher und leistungsstärker.
Unser Leistungs­ver­sprechen
24 Stunden Service
Rund um die Uhr beweisen wir unsere Zuverlässigkeit.
mehr lesen
Made in Germany
Um Ihnen die größtmögliche Produktqualität zu garantieren...
mehr lesen
Starke Zusatzleistungen
So umfangreich wie unser Sortiment sind auch unsere Anarbeitungs­möglich­keiten.
mehr lesen
Breite Produktpalette
Unser Sortiment ist bekannt für seine Breite und Tiefe.
mehr lesen
Persönliche Betreuung
Jeder unserer Mitarbeiter kennt unsere Produkte ganz genau.
mehr lesen
Zentrales Lager
Damit wir Sie jederzeit und kurzfristig zufriedenstellen können.

these descriptions are not really useful 

In [12]:
import RAKE
rake = RAKE.Rake(RAKE.SmartStopList())
rake.run(text, minCharacters = 1, maxWords = 5, minFrequency = 1)

[('der beratung und den arbeitsabläufen', 24.5),
 ('wir freuen uns auf sie', 23.0),
 ('mehr lesen\nstarke zusatzleistungen', 15.2),
 ('mehr lesen\nqualität\nkontinuität', 15.2),
 ('mehr lesen\nkontakt gefällig', 15.2),
 ('hoberg und driesch gmbh &', 14.5),
 ('erreichen sie uns', 11.0),
 ('mehr lesen\nmade', 10.2),
 ('mehr lesen\nflexibel', 10.2),
 ('hoberg & driesch', 6.0),
 ('kg 2018\ntop', 4.0),
 ('termintreu', 1.0),
 ('persönlich', 1.0)]

In [13]:
import pyperclip
pyperclip.copy(text)


## extracting metadata

In [2]:
import pandas as pd
from urllib.request import urlopen
headers_dom = ['url', 'section', 'code']
dom = pd.read_csv('../data/raw/20181018 Domain+WZ2008 Hackathon INOBAS.csv', sep=';', names=headers_dom, encoding='latin-1', skiprows=1)

dom.head()

,url,section,code
0,www.hoberg-driesch.de,L,68.31
1,www.hgdf.de,K,64.20
2,www.gasag.de,D,35.30
3,www.ergodirekt.de,K,65.11
4,www.wahler.de,M,70.10


In [3]:
headers = {'User-Agent': 
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Safari/65.1.15'}    

In [4]:
from urllib.request import urlopen, Request
def extract_metadata(url):
    print("normal:" + url)
    abstr = ""
    desc = ""
    keywords = ""
    crashed = False
    try:
        try:
            content = urlopen(Request("https://" + url, headers={'User-Agent': 'Mozilla'}), timeout = 10)
        except:
            try:
                content = urlopen(Request("http://" + url, headers={'User-Agent': 'Mozilla'}), timeout = 10)
            except:
                raise
        soup = BeautifulSoup(content, 'html.parser')
        abstr = ""
        desc = ""
        keywords = ""
        for meta in soup.findAll("meta"):
            if not meta.has_attr('content'):
                continue
            metaname = meta.get('name', '').lower()
            metaprop = meta.get('property', '').lower()
            if 'description' == metaname or metaprop.find("description")>0:
                desc = meta['content'].strip()
            if 'abstract' == metaname or metaprop.find("abstract")>0:
                abstr = meta['content'].strip()
            if 'keywords' == metaname or metaprop.find("keywords")>0:
                keywords = meta['content'].strip()
    except Exception as e: 
            print(e)
            print(url)
            crashed = True
    return (abstr, desc, keywords, crashed)

In [5]:
extract_metadata("www.hoberg-driesch.de")

normal:www.hoberg-driesch.de


('Qualitätsstahlrohre von Hoberg & Driesch. Von nahtlos bis grenzenlos.',
 'Hoberg & Driesch GmbH & Co. KG ist eines der führenden europäischen Großhandelshäuser für Stahlrohre mit Firmensitz in Düsseldorf und weiteren Niederlassungen in Hamburg, Kornwestheim, München, Dresden und Istanbul.',
 'Röhrengrosshandel, Starkwandrohre, Normalwandrohre, Gewinderohre, Kesselrohre, Hohlprofile, Präzisions\xadstahl\xadrohre, HPZ-Rohre, Wälzlager\xadstahl\xadrohre, Feinkornbaustahlrohre, Maschinenbaustahlrohre',
 False)

In [ ]:
import swifter

res = pd.DataFrame(dom["url"])
res[["abstract", "description", "keywords", "crashed"]] = res['url'].swifter.apply(extract_metadata).apply(pd.Series)

0        normal:www.hoberg-driesch.de
1                  normal:www.hgdf.de
2                 normal:www.gasag.de
3            normal:www.ergodirekt.de
4                normal:www.wahler.de
5              normal:www.msa-auer.de
6           normal:www.nuernberger.de
7                 normal:www.ds-cc.de
8            normal:www.mlk-berlin.de
9             normal:www.goldhofer.de
10        normal:www.faber-castell.de
11    normal:www.klinikum-landshut.de
12              normal:www.biotest.de
13           normal:www.vattenfall.de
14               normal:www.klksig.de
15           normal:www.dsv-gruppe.de
Name: url, dtype: object
<urlopen error unknown url type: 0        http>
0        www.hoberg-driesch.de
1                  www.hgdf.de
2                 www.gasag.de
3            www.ergodirekt.de
4                www.wahler.de
5              www.msa-auer.de
6           www.nuernberger.de
7                 www.ds-cc.de
8            www.mlk-berlin.de
9             www.goldhofer.de
10       

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


normal:www.vb-alzey-worms.de
normal:www.westpfalz-klinikum.de
normal:www.kob.de
normal:www.impact-group.de
normal:www.m-e-v.de
normal:www.treureal-service.de
normal:www.theresienkrankenhaus.de
normal:www.grimminger.de
normal:www.heidenreich-gruppe.de
normal:www.firmengruppe-heidenreich.de
normal:www.friatec.de
normal:www.lieblang.de
normal:www.hauck-hygiene.de
<urlopen error [Errno 54] Connection reset by peer>
www.hauck-hygiene.de
normal:www.swhd.de
normal:www.uni-heidelberg.de
normal:www.ekosem-agrar.de
normal:www.wild.de
normal:www.ndw-drahtundstahl.de
normal:www.langer-service.de
normal:www.naturin.de
normal:www.klett.de
normal:www.suedwestbank.de
normal:www.hallesche.de
normal:www.vinzenz.de
normal:www.hafenstuttgart.de
normal:www.kodak.de
normal:www.wolff-mueller.de
normal:www.woerwagpharma.de
normal:www.diako-mannheim.de
normal:www.bauder.de
normal:www.ssb-ag.de
normal:www.knv.de
normal:www.hansa.de
normal:www.swmh.de
normal:www.ga-eas.de
normal:www.tgs.de
normal:www.adcapital.d

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


normal:www.vb-alzey-worms.de
normal:www.westpfalz-klinikum.de
normal:www.kob.de
normal:www.impact-group.de
normal:www.m-e-v.de
normal:www.treureal-service.de
normal:www.theresienkrankenhaus.de
normal:www.grimminger.de
normal:www.heidenreich-gruppe.de
normal:www.firmengruppe-heidenreich.de
normal:www.friatec.de
normal:www.lieblang.de
normal:www.hauck-hygiene.de
<urlopen error [Errno 54] Connection reset by peer>
www.hauck-hygiene.de
normal:www.swhd.de
normal:www.uni-heidelberg.de
normal:www.ekosem-agrar.de
normal:www.wild.de
normal:www.ndw-drahtundstahl.de
normal:www.langer-service.de
normal:www.naturin.de
normal:www.klett.de
normal:www.suedwestbank.de
normal:www.hallesche.de
normal:www.vinzenz.de
normal:www.hafenstuttgart.de
normal:www.kodak.de
normal:www.wolff-mueller.de
normal:www.woerwagpharma.de
normal:www.diako-mannheim.de
normal:www.bauder.de
normal:www.ssb-ag.de
normal:www.knv.de
normal:www.hansa.de
normal:www.swmh.de
normal:www.ga-eas.de
normal:www.tgs.de
normal:www.adcapital.d

In [ ]:
res["crashed"].value_counts()

In [91]:
content = urlopen("https://www.conrad.de")

HTTPError: HTTP Error 403: Forbidden